In [1]:
import numpy as np
import matplotlib.pyplot as plt
import twobodypropogator

# Uncertainty propagation methods - Monte Carlo (MC) Simulation

[Y.-Z. Luo και Z. Yang, ‘A review of uncertainty propagation in orbital mechanics’, Progress in Aerospace Sciences, τ. 89, σσ. 23–39, 2017.](https://doi.org/10.1016/j.paerosci.2016.12.002)

This brute force approach allows a bench-mark for other models. Samples of an initial probability distribution for each trajectory is recorded and propogated using a numerical ODE solver. The mean and covariance matrix for each time-step is simulated.

## Initial States DOE

Generate $N$ initial conditions for a satellite, $\{\vec{x}_0^{(i)}\in \mathbb{R}^6: i\in[1, N]\subset\mathbb{N}^N\}$, 

where ${\vec{x}_0}^{(i)}=\begin{matrix}[\vec{r}^{(i)}, & \vec{v}^{(i)}]^\top\end{matrix}$ is the initial state for a satellite on a trajectory and $\vec{r}$ and $\vec{v}$ are the position and velocity vectors respectively in the central body's inertial Cartesian reference frame.

The determnisitc initial $N$ states are generated using a random latin-hypercube DOE of 6 keplerian elements in $\mathbb{R}^{N\times 6}$ with bounds:

- $a\in[1000, 2000]$
- $e\in[0, 0.5]$
- $i\in[0, \pi]$
- $\varOmega\in[0, 2\pi]$
- $\omega\in[0, 2\pi]$
- $\nu\in[0, 2\pi]$

The initial cartesian states in $\mathbb{R}^{N\times 6}$ are calculated from the keplerian elements DOE.

In [2]:
np.random.seed(123)
N_ICS = 300

a_bounds = [1000, 2000]  # semi-major axis [km]
e_bounds = [0, 0.5]  # eccentricity [-]
i_bounds = [0, np.pi]  # inclination [rad]
Omega_bounds = [0, 2*np.pi]  # right ascension [rad]
argp_bounds = [0, 2*np.pi]  # argument of periapsis [rad]
nu_bounds = [0, 2*np.pi]  # true anomaly [rad]

keplerian_bounds = np.asarray(
    [a_bounds, e_bounds, i_bounds, Omega_bounds, argp_bounds, nu_bounds]
)

cart_ics, kepler_ics = twobodypropogator.keplerian_ics_doe(N_ICS, keplerian_bounds)
print(kepler_ics.shape)


(300, 6)


# Initial Probability Distribution

[C. Sabol, K. Hill, K. Alfriend, T. Sukut, Nonlinear effects in the correlation of tracks and covariance propagation, Acta Astronaut. 84 (2013) 69–80](https://doi.org/10.1016/j.actaastro.2012.08.023)

The orbital elements observations are corrupted with Gaussian noise.

The noise standard deviations are 30 m and 36 arcsec for the range and angles, respectively, with the intent of simulating errors representative of space surveillance radar systems.

$m$ initial trajectories are sampled from the initial probability distribution.

In [3]:
range_err = 30e-3  # [km]
angle_err = 36*4.84814e-6  # [rad]

# dictionary and distribution type is passed as an argument later
noise_std = {
    'a': range_err, 'e': range_err, 'i': angle_err,
    'Omega': angle_err, 'argp': angle_err, 'nu': angle_err
}

distribution_type = 'normal'


## Perform Simulation

Generate $N$ `<MonteCarloPropogator>` class instances (one for each trajectory).

Propogate samples of each initial probability distribution. The class will calculate the first and second statistical moments for each epoch.

In [6]:
from time import time

np.random.seed(123)
T0 = 0
TF = 3600
DT = 5
epochs = np.arange(T0, TF, DT)

m_samples = 500  # number of samples

monte_carlo_sims = []
tic = time()
for i, state0_nominal in enumerate(cart_ics):
    mc_sim = twobodypropogator.MonteCarloPropogator(
        state0=state0_nominal,
        t_span=[T0, TF],
        n_samples=m_samples,
        initial_pdf_type=distribution_type,
        initial_noise_std=noise_std,
        central_body='earth'
    )
    mc_sim.propogate(
        t_eval=epochs,
        method='RK45',
        max_step=DT,
        tol=1e-10
    )
    monte_carlo_sims.append(mc_sim)

    # progress bar for output
    eta = round((time()-tic) * (N_ICS/(i+1) - 1)/60, 2)
    print(
        f'\rCompleted Trajectory : {i+1}/{N_ICS}\tETA [mins] : {eta}', end='\r')
print(f'SIMULATION COMPLETE                                     ', end='\n')

monte_carlo_sims = np.array(monte_carlo_sims, dtype=object)

# save .npy array of objects
file_prefix = 'E:\\simulations\\data'
file_path = f'{file_prefix}\\monte_carlo_sims_{N_ICS}ics_{m_samples}smp.npy'
np.save(file_path, monte_carlo_sims)
print(f'Output written to : "{file_path}"', end='\n')


SIMULATION COMPLETE                                     
Output written to : "E:\simulations\data\monte_carlo_sims_300ics_500smp.npy"


In [2]:
N_ICS = 300
m_samples = 500

file_prefix = 'E:\\simulations\\data'
file_path = f'{file_prefix}\\monte_carlo_sims_{N_ICS}ics_{m_samples}smp.npy'
monte_carlo_sims = np.load(file_path, allow_pickle=True)


## Train, Validation and Test Data Split

In [39]:
from sklearn.model_selection import train_test_split
means_tensor = np.asarray([mc_sim.means for mc_sim in monte_carlo_sims])
covs_tensor = np.asarray([mc_sim.means for mc_sim in monte_carlo_sims])


In [263]:
means_training, means_testing = train_test_split(means_tensor)
means_testing = np.split(means_testing, 3)
means_learning, means_validation = train_test_split(means_training)
means_learning = np.split(means_learning, 14)


In [264]:
for train_num, means_train_set in enumerate(means_learning): 
    data_prefix = 'TwoBodyMC'
    data_set = f'train{train_num+1}_x'
    np.save(f'data\\{data_prefix}_{data_set}.npy', means_train_set)

np.save(f'data\\{data_prefix}_val_x.npy', means_validation)

for test_num, means_test_set in enumerate(means_testing):
    data_prefix = 'TwoBodyMC'
    data_set = f'test{test_num+1}_x'
    np.save(f'data\\{data_prefix}_{data_set}.npy', means_test_set)
